In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os, shutil, sys, re
from scipy import stats
from tabulate import tabulate
import math
import matplotlib.patches as mpatches

from matplotlib.lines import Line2D

## Defining data files and parameters

Define directories where the data is stored and where thew results (plots) are to be written to, along with experimental parameters.

In [2]:
PRNG_dir = "../experimentData/PointDistributionData/CollectedData_uniformity/PRNG/"
QRNG_dir = "../experimentData/PointDistributionData/CollectedData_uniformity/QRNG/"

precision =  'double'#'single'#

alpha = 0.05 # significance
NSeeds = 50

Results_directory = "../CollectedData_uniformity/Plots"
Table_digits = '.4f'


Nsamples = ['1K', '5K', '10K', '50K']
Nsamples_number = np.array([1000, 5000, 10000, 50000])

OneSample = '50K'


## Reading the data

In [3]:
def parseResults(filename_LES_statistics:str, filename_NN_statistics:str):
    results = {}

    data = np.genfromtxt(filename_LES_statistics, delimiter=',')
    results['meanLES'] = data[0]
    results['varLES'] = data[1]
    
    data = np.genfromtxt(filename_NN_statistics, delimiter=',')
    results['meanNN'] = data[0]
    results['varNN'] = data[1]
    
    return results


In [4]:
def parseResults_mean_var(filename_data:str):
    data = np.genfromtxt(filename_data, delimiter=',')
    results = {}
    results['meanQ'] = np.mean(data)
    results['varQ'] = np.var(data)
    
    return results


In [5]:
if precision=='double':
    ResultFilenameTemplate_les = "vis_{Samples}/lesStatistics_double_{Nrep:d}.dat"
    ResultFilenameTemplate_nn = "vis_{Samples}/nnStatistics_double_{Nrep:d}.dat"
    ResultFilenameTemplate_q = "vis_{Samples}/nndata_PE_Q_double_{Nrep:d}.dat"
    ResultFilenameTemplate_CoV = "vis_{Samples}/nndata_CoV_double_{Nrep:d}.dat"
    ResultFilenameTemplate_Avg = "vis_{Samples}/nndata_Avg_double_{Nrep:d}.dat"
    ResultFilenameTemplate_Rm = "vis_{Samples}/nndata_Rm_double_{Nrep:d}.dat"
elif precision=='single':
    ResultFilenameTemplate_les = "vis_{Samples}/lesStatistics_single_{Nrep:d}.dat"
    ResultFilenameTemplate_nn = "vis_{Samples}/nnStatistics_single_{Nrep:d}.dat"
    ResultFilenameTemplate_q = "vis_{Samples}/nndata_PE_Q_single_{Nrep:d}.dat"
    ResultFilenameTemplate_CoV = "vis_{Samples}/nndata_CoV_single_{Nrep:d}.dat"
    ResultFilenameTemplate_Avg = "vis_{Samples}/nndata_Avg_single_{Nrep:d}.dat"
    ResultFilenameTemplate_Rm = "vis_{Samples}/nndata_Rm_single_{Nrep:d}.dat"


# Define array names
array_names = ['qrng_les_means', 'qrng_les_variances', 'prng_les_means', 'prng_les_variances',
               'qrng_nn_means', 'qrng_nn_variances', 'prng_nn_means', 'prng_nn_variances',
               'qrng_q_means', 'qrng_q_variances', 'prng_q_means', 'prng_q_variances',
               'qrng_Avg_means', 'qrng_Avg_variances', 'prng_Avg_means', 'prng_Avg_variances',
               'qrng_CoV_means', 'qrng_CoV_variances', 'prng_CoV_means', 'prng_CoV_variances',
               'qrng_Rm_means', 'qrng_Rm_variances', 'prng_Rm_means', 'prng_Rm_variances',
               'qrng_CE', 'prng_CE', 'qrng_R_nr', 'prng_R_nr']

# Initialize arrays using a loop
arrays = {name: np.zeros((NSeeds, len(Nsamples)), dtype=float) for name in array_names}

#iterate over the batch sizes
for repIdx in range(NSeeds):
    for Sample, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
        #print("NSeeds", repIdx, "Sample",Sample, "SampleIdx", nSamplesIdx)
        filenameParameters= {'Samples':Sample, 'Nrep':repIdx+1}
        filename_les = ResultFilenameTemplate_les.format(**filenameParameters)
        filename_nn = ResultFilenameTemplate_nn.format(**filenameParameters)
        filename_q = ResultFilenameTemplate_q.format(**filenameParameters)
        filename_Avg = ResultFilenameTemplate_Avg.format(**filenameParameters)
        filename_Rm = ResultFilenameTemplate_Rm.format(**filenameParameters)
        filename_CoV = ResultFilenameTemplate_CoV.format(**filenameParameters)
        
        data = parseResults(QRNG_dir +filename_les, QRNG_dir +filename_nn)
        
        data_q = parseResults_mean_var(QRNG_dir +filename_q)
        data_Avg = parseResults_mean_var(QRNG_dir +filename_Avg)
        data_Rm = parseResults_mean_var(QRNG_dir +filename_Rm)
        data_CoV = parseResults_mean_var(QRNG_dir +filename_CoV)

        data_dicts_qrng = {'qrng_q': data_q, 'qrng_Rm': data_Rm, 'qrng_Avg': data_Avg, 'qrng_CoV': data_CoV}
        
        #distribute data to the arrays
                
        arrays['qrng_les_means'][repIdx, nSamplesIdx] = data['meanLES']
        arrays['qrng_les_variances'][repIdx, nSamplesIdx] = data['varLES']
        
        arrays['qrng_nn_means'][repIdx, nSamplesIdx] = data['meanNN']
        arrays['qrng_nn_variances'][repIdx, nSamplesIdx] = data['varNN']
        
        for prefix, data in data_dicts_qrng.items():
            arrays[f"{prefix}_means"][repIdx, nSamplesIdx] = data['meanQ']
            arrays[f"{prefix}_variances"][repIdx, nSamplesIdx] = data['varQ']
        
        
        #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
        ##                          PRNG
        #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

        data = parseResults(PRNG_dir + filename_les, PRNG_dir + filename_nn)
        
        data_q = parseResults_mean_var(PRNG_dir +filename_q)
        data_Avg = parseResults_mean_var(PRNG_dir +filename_Avg)
        data_Rm = parseResults_mean_var(PRNG_dir +filename_Rm)
        data_CoV = parseResults_mean_var(PRNG_dir +filename_CoV)
        
        data_dicts_prng = {'prng_q': data_q, 'prng_Rm': data_Rm, 'prng_Avg': data_Avg, 'prng_CoV': data_CoV}

        #distribute data to the arrays
        arrays['prng_les_means'][repIdx, nSamplesIdx] = data['meanLES']
        arrays['prng_les_variances'][repIdx, nSamplesIdx] = data['varLES']
        
        arrays['prng_nn_means'][repIdx, nSamplesIdx] = data['meanNN']
        arrays['prng_nn_variances'][repIdx, nSamplesIdx] = data['varNN']
        
        for prefix, data in data_dicts_prng.items():
            arrays[f"{prefix}_means"][repIdx, nSamplesIdx] = data['meanQ']
            arrays[f"{prefix}_variances"][repIdx, nSamplesIdx] = data['varQ']
        

/mnt/TGT/DatavisVENV_V2/lib/python3.8/site-packages/numpy/core/_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


#### Functions to create plots and statistics tables

In [6]:
def boxplot_statistic(Nsamples_number, qrng_data, prng_data, test_type,precision,test):
    fig, axes = plt.subplots(1, len(Nsamples_number), figsize=(12, 5))
    for Sample, nSamplesIdx in zip(Nsamples_number, range(len(Nsamples_number)) ):
        bp = axes[nSamplesIdx].boxplot([qrng_data[:, nSamplesIdx], prng_data[:, nSamplesIdx]], patch_artist=True, showmeans=True, meanline=True, medianprops=dict(color='red'))
        colors = ['blue', 'orange']
        for box, color in zip(bp['boxes'], colors):
            box.set(color=color, linewidth=2)
            box.set(facecolor='lightgrey')        
        axes[nSamplesIdx].set_title('${}$ test, {} points'.format(test_type, Sample))

        axes[nSamplesIdx].set_xticks([])
    
    legend_colors = ['blue', 'orange']
    legend_labels = ['QRNG', 'PRNG']

    # Creating legend patches
    legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]
    legend_elements = [Line2D([0], [0], color='red', lw=2, label='Median'), 
                    Line2D([0], [0], color='green', lw=2, linestyle='--', label='Mean')]
    all_handles = legend_elements + legend_patches
    plt.legend(handles=all_handles, loc='upper center', fontsize='small', bbox_to_anchor=(0.5, 0.0), ncol=4)
    plt.tight_layout()
    
    # Construct the file path
    file_path = os.path.join(Results_directory, test + '_'+precision+'.png')
    plt.savefig(file_path, dpi=150, bbox_inches='tight')
    plt.show()


In [7]:
# T-test
def T_test(case1_means,prngcase1_means):
    PoolVariance = 1.0 / (len(case1_means) + len(prngcase1_means) - 2) * (
                (len(case1_means) - 1) * np.std(case1_means, ddof=1) ** 2 + (
                    len(prngcase1_means) - 1) * np.std(prngcase1_means, ddof=1) ** 2)
    t_samples = (np.mean(case1_means) - np.mean(prngcase1_means)) / (
                np.sqrt(PoolVariance * (1.0 / len(prngcase1_means) + 1.0 / len(case1_means))))
    tDOF_joint = len(case1_means) + len(prngcase1_means) - 2
    t_1_joint = stats.t.ppf(alpha / 2, tDOF_joint)
    t_2_joint = stats.t.ppf(1 - alpha / 2, tDOF_joint)
    
    # Calculate the p-value for a two-tailed test
    p_value = 2 * min(stats.t.cdf(t_samples, tDOF_joint), 1 - stats.t.cdf(t_samples, tDOF_joint))
    
    # Determine the test result based on the p-value
    if t_samples < t_1_joint:
        test_result_T_test = 'Significant'
    elif t_samples > t_2_joint:
        test_result_T_test = 'Significant'
    else:
        test_result_T_test = 'Not Significant'
    return t_samples, p_value, test_result_T_test, t_1_joint,t_2_joint

In [8]:
def F_test(case1_means,prngcase1_means):
    f = np.var(case1_means, ddof=1) / np.var(prngcase1_means, ddof=1)
    f_1 = stats.f.ppf(alpha / 2, len(case1_means) - 1, len(prngcase1_means) - 1)
    f_2 = stats.f.ppf(1 - alpha / 2, len(case1_means) - 1, len(prngcase1_means) - 1)
    if f < f_1:
        test_result_F_test = 'Significant'
    elif f > f_2:
        test_result_F_test = 'Significant'
    else:
        test_result_F_test = 'Not Significant'
    
    
    p_value_f = stats.f.sf(f, len(case1_means) - 1, len(prngcase1_means) - 1)
    return f, p_value_f, test_result_F_test, f_1, f_2

    #----- For a two-tailed interpretation, you might double the p-value (not typical for F-tests focusing on variance comparisons):
    #----- p_value_f_two_tailed = 2 * min(p_value_f, 1 - p_value_f)

    #p_value_f_two_tailed = 2 * min(p_value_f, 1 - p_value_f)
    #return f, p_value_f_two_tailed, test_result_F_test, f_1, f_2

In [9]:
def run_statistical_tests(test,precision, Nsamples, qrng_means, prng_means,alpha):
    # Example data (replace this with your own data)
    data_Wlcx = {
        'Sample Size': [],
        'Wilcoxon Statistic': [],
        'P-value': [],
        'Test Result': []  
    }

    data_Ttest = {
        'Sample Size': [],
        'Test statistic': [],
        'P-value': [],
        'Test Result': []  
    }

    data_Ftest = {
        'Sample Size':[],
        'Test statistic': [],
        'P-value': [],
        'Test Result': []  
    }

    data_KStest = {
        'Sample Size': [],
        'Test statistic': [],
        'P-value': [],
        'Test Result': []  
    }

    data_ADtest = {
        'Sample Size': [],
        'Test statistic': [],
        'P-value': [],
        'Test Result': []  
    }
    
    data_ALL_Tests = {
        'W test':[],
        'T test': [],
        'F test': [],
        'AD test': [],
        'KS test': []
    }

    Sample_as_number = Nsamples
    
    print('alpha=',alpha)

    for Sample, nSamplesIdx in zip(Nsamples, range(len(Nsamples))):
        case1_means = qrng_means[:, nSamplesIdx]
        prngcase1_means = prng_means[:, nSamplesIdx]

        statistic, p_value = stats.wilcoxon(case1_means, prngcase1_means)
        # Append results to the data dictionary
        data_Wlcx['Sample Size'].append(Sample_as_number[nSamplesIdx])

        data_Wlcx['Wilcoxon Statistic'].append(statistic)
        data_Wlcx['P-value'].append(format(p_value, Table_digits))

        # Determine the test result based on the p-value
        test_result_Wlcx = 'Significant' if p_value < alpha else 'Not Significant'
        data_Wlcx['Test Result'].append(test_result_Wlcx)

        # T-test
        [t_samples, p_value, test_result_T_test,t_1_joint,t_2_joint] = T_test(case1_means,prngcase1_means)
        
        data_Ttest['P-value'].append(format(p_value, Table_digits))
        data_Ttest['Test statistic'].append(format(t_samples, Table_digits))
        data_Ttest['Sample Size'].append(Sample_as_number[nSamplesIdx])
        data_Ttest['Test Result'].append(test_result_T_test)

        # F-test        
        [f, p_value_f, test_result_F_test, f_1, f_2] = F_test(case1_means,prngcase1_means)
        data_Ftest['Sample Size'].append(Sample_as_number[nSamplesIdx])
        data_Ftest['Test statistic'].append(format(f, Table_digits))
        data_Ftest['P-value'].append(format(p_value_f, Table_digits))
        data_Ftest['Test Result'].append(test_result_F_test)

        # Kolmogorov-Smirnov test
        ks_result = stats.ks_2samp(case1_means, prngcase1_means)
        ks_statistic = ks_result.statistic
        p_value = ks_result.pvalue
        test_result_KS = 'Significant' if p_value < alpha else 'Not Significant'

        data_KStest['P-value'].append(format(p_value, Table_digits))
        data_KStest['Sample Size'].append(Sample_as_number[nSamplesIdx])
        data_KStest['Test statistic'].append(format(p_value, Table_digits))
        data_KStest['Test Result'].append(test_result_KS)

        # Anderson-Darling test
        ad_result = stats.anderson_ksamp([case1_means, prngcase1_means])
        ad_statistic = ad_result.statistic
        p_value = ad_result.pvalue
        test_result_AD = 'Significant' if p_value < alpha else 'Not Significant'
        
        data_ADtest['P-value'].append(format(p_value, Table_digits))
        data_ADtest['Sample Size'].append(Sample_as_number[nSamplesIdx])
        data_ADtest['Test statistic'].append(format(p_value, Table_digits))
        data_ADtest['Test Result'].append(test_result_AD)
        
        #==================================================================
        data_ALL_Tests['W test'].append(data_Wlcx['P-value'][-1])
        data_ALL_Tests['KS test'].append(data_KStest['P-value'][-1])
        data_ALL_Tests['T test'].append(data_Ttest['P-value'][-1])
        data_ALL_Tests['F test'].append(data_Ftest['P-value'][-1])
        data_ALL_Tests['AD test'].append(data_ADtest['P-value'][-1])
        

    print(test, precision, "|    Wilcoxon test")
    print('Wilcoxon test is the sum of the ranks of the positive (or negative) differences. It represents the degree of difference between the two groups')
    table_W = tabulate(data_Wlcx, headers='keys', tablefmt='pretty')
    print(table_W)
    
    print(test, precision, "|    T-test")
    print([t_1_joint,t_2_joint])
    table_T = tabulate(data_Ttest, headers='keys', tablefmt='pretty')
    print(table_T)
    
    print(test, precision, "|    Anderson-Darling test")
    table_AD = tabulate(data_ADtest, headers='keys', tablefmt='pretty')
    print(table_AD)
    
    print(test, precision, "|    F-test")
    print([f_1,f_2])
    table_F = tabulate(data_Ftest, headers='keys', tablefmt='pretty')
    print(table_F)
    
    print(test, precision, "|    Kolmogorov-Smirnov test")
    table_KS = tabulate(data_KStest, headers='keys', tablefmt='pretty')
    print(table_KS)
    
    # Print results as a table
    if not os.path.exists(Results_directory):
        os.makedirs(Results_directory)
    file_path = Results_directory +'/'+ test+'_'+precision+'_table.txt'
    
    with open(file_path, 'w') as file:
        file.write(f"alpha = {alpha}\n")        
        file.write(f"{test}, {precision} |    Wilcoxon test\n")        
        file.write(table_W)
        file.write('\n') 
    with open(file_path, 'a') as file:
        file.write(f"{test}, {precision} |    T-test\n")        
        file.write(f"[{t_1_joint},{t_2_joint}]\n")        
        file.write(table_T)
        file.write('\n') 
        file.write(f"{test}, {precision} |    Anderson-Darling test\n")        
        file.write(table_AD)
        file.write('\n')  
        file.write(f"{test}, {precision} |    F-test\n")        
        file.write(f"[{f_1},{f_2}]\n")        
        file.write(table_F)
        file.write('\n')  
        file.write(f"{test}, {precision} |     Kolmogorov-Smirnov test\n")        
        file.write(table_KS)
        file.write('\n')  # Add an extra line 
        
    return data_ALL_Tests

In [10]:
def statistics_results(test,alpha,precision):
    ALL_tests_for_metric={}
    if test=='les':
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_les_means'], arrays['prng_les_means'],alpha)
        fig, axes = plt.subplots(1, len(Nsamples_number), figsize=(12, 5))
        for Sample, nSamplesIdx in zip(Nsamples_number, range(len(Nsamples_number)) ):
            bp = axes[nSamplesIdx].boxplot([np.pi*(arrays['qrng_les_means'][:, nSamplesIdx])**2, np.pi*(arrays['prng_les_means'][:, nSamplesIdx])**2],patch_artist=True, showmeans=True, meanline=True, medianprops=dict(color='red'))
            n = float(Nsamples_number[nSamplesIdx])
            test_line = (-math.log(-math.log(1 - alpha)) + math.log(n) + math.log(math.log(n)))/n
            axes[nSamplesIdx].axhline(y=test_line, color='r', linestyle='--', label='Test Line')
            colors = ['blue', 'orange']
            for box, color in zip(bp['boxes'], colors):
                box.set(color=color, linewidth=2)
                box.set(facecolor='lightgrey')        
            axes[nSamplesIdx].set_title(f'Janson test for LES, {Sample} points')
            axes[nSamplesIdx].set_xticks([])
        legend_colors = ['blue', 'orange']
        legend_labels = ['QRNG', 'PRNG']
        legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]
        legend_elements = [Line2D([0], [0], color='red', lw=2, label='Median'), 
                        Line2D([0], [0], color='green', lw=2, linestyle='--', label='Mean')]
        all_handles = legend_elements + legend_patches
        plt.legend(handles=all_handles, loc='upper center', fontsize='small', bbox_to_anchor=(0.5, 0.0), ncol=4)
        plt.tight_layout()
        
        file_path = os.path.join(Results_directory, 'Janson_' + precision+'.png')
        plt.savefig(file_path, dpi=150)
        plt.show()
        
        boxplot_statistic(Nsamples_number,arrays['qrng_les_means'], arrays['prng_les_means'], 'LES',precision,test)
        
    elif test=='nn':
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_nn_means'], arrays['prng_nn_means'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_nn_means'], arrays['prng_nn_means'], 'NN',precision,test)
    elif test=='PE_Q':
        ALL_tests_for_metric =run_statistical_tests(test,precision, Nsamples, arrays['qrng_q_means'], arrays['prng_q_means'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_q_means'], arrays['prng_q_means'], 'Q',precision,test)

    elif test=='Avg':
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_Avg_means'], arrays['prng_Avg_means'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_Avg_means'], arrays['prng_Avg_means'], r'\bar\gamma',precision,test)

    elif test=='CoV':
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_CoV_means'], arrays['prng_CoV_means'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_CoV_means'], arrays['prng_CoV_means'], 'C_V',precision,test)

    elif test=='Rm':
        #run_statistical_tests(test,precision, Nsamples, arrays['qrng_Rm_means'], arrays['prng_Rm_means'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_Rm_means'], arrays['prng_Rm_means'], 'R_m',precision,test)
        
    elif test =='CE':
        for Sample, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
            n = float(Nsamples_number[nSamplesIdx])
            E_gamma = 0.5 * pow(n, -1/2) + 4 * pow(n, -1) * (0.514 + 0.412 * pow(n, -1/2))
            var_gamma = 0.070 * pow(n, -2) + 0.148 * pow(n, -5/2)
            arrays['qrng_CE'][:, nSamplesIdx] =  (arrays['qrng_Avg_means'][:, nSamplesIdx] - E_gamma)/np.sqrt(var_gamma)
            arrays['prng_CE'][:, nSamplesIdx] =  (arrays['prng_Avg_means'][:, nSamplesIdx] - E_gamma)/np.sqrt(var_gamma)
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_CE'], arrays['prng_CE'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_CE'], arrays['prng_CE'], 'CE',precision,test)

            
    elif test =='R_nr':
        for Sample, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
            n = float(Nsamples_number[nSamplesIdx])
            E_gamma = 0.5 * pow(n, -1/2) + 4 * pow(n, -1) * (0.514 + 0.412 * pow(n, -1/2))
            arrays['qrng_R_nr'][:, nSamplesIdx] =  (arrays['qrng_Avg_means'][:, nSamplesIdx])/E_gamma
            arrays['prng_R_nr'][:, nSamplesIdx] =  (arrays['prng_Avg_means'][:, nSamplesIdx])/E_gamma
        
        ALL_tests_for_metric = run_statistical_tests(test,precision, Nsamples, arrays['qrng_R_nr'], arrays['prng_R_nr'],alpha)
        boxplot_statistic(Nsamples_number,arrays['qrng_R_nr'], arrays['prng_R_nr'], 'R_{nr}',precision,test)
    return ALL_tests_for_metric
        

In [11]:
test_list = {'nn','PE_Q','CoV','CE','R_nr','les'} ##test_list = {'les','nn','PE_Q','CoV','Rm','CE','R_nr','les'}

Dictionary_General={}
for test in test_list:
    ALL_tests_for_metric_tmp = statistics_results(test,alpha,precision)
    Dictionary_General[test] = ALL_tests_for_metric_tmp

alpha= 0.05


/tmp/ipykernel_24508/2011340280.py:92: UserWarning: p-value capped: true value larger than 0.25
  ad_result = stats.anderson_ksamp([case1_means, prngcase1_means])


AttributeError: 'Anderson_ksampResult' object has no attribute 'pvalue'

## All test results for a specific sample size

In [ ]:
#OneSample = "50K"
nSamplesIdx = Nsamples.index(OneSample)


#test_list = ['NN','Q','C_V','CE','R_{nr}'] 
test_list = ['NN','LES','Q','CE','R_{nr}'] 


QRNG_data_keys = ['qrng_nn_means', 'qrng_q_means', 'qrng_CoV_means', 'qrng_CE', 'qrng_R_nr']
PRNG_data_keys = ['prng_nn_means', 'prng_q_means', 'prng_CoV_means', 'prng_CE', 'prng_R_nr']

fig, axes = plt.subplots(1, len(test_list), figsize=(12, 5))

for testIdx, (qrng_key, prng_key) in enumerate(zip(QRNG_data_keys, PRNG_data_keys)):
    qrng_data = arrays[qrng_key]  
    prng_data = arrays[prng_key]  
    bp = axes[testIdx].boxplot([qrng_data[:, nSamplesIdx], prng_data[:, nSamplesIdx]], patch_artist=True, showmeans=True, meanline=True, medianprops=dict(color='red'))
    colors = ['blue', 'orange']
    for box, color in zip(bp['boxes'], colors):
            box.set(color=color, linewidth=2)
            box.set(facecolor='lightgrey')        

    axes[testIdx].set_title('${}$ test'.format(test_list[testIdx]))
    axes[testIdx].set_xticks([])
    
legend_colors = ['blue', 'orange']
legend_labels = ['QRNG', 'PRNG']
legend_patches = [mpatches.Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]
legend_elements = [Line2D([0], [0], color='red', lw=2, label='Median'), 
                Line2D([0], [0], color='green', lw=2, linestyle='--', label='Mean')]
all_handles = legend_elements + legend_patches
fig.subplots_adjust(bottom=0.1)  
fig.legend(handles=all_handles, loc='lower center', fontsize='small', ncol=4, frameon=False, bbox_to_anchor=(0.5, -0.04))
fig.text(0.01, -0.02, f"{Nsamples_number[nSamplesIdx]} sample points of {precision} precision", fontsize=10, ha='left', va='bottom')

plt.tight_layout()

file_path = os.path.join(Results_directory, f'Test_summary_for_'+OneSample+'_'+precision+'_points.png')
plt.savefig(file_path, dpi=150, bbox_inches='tight')

plt.show()


In [ ]:
rows = ['nn', 'PE_Q', 'CoV', 'CE', 'R_nr','les']
columns = ['W test', 'T test', 'F test', 'AD test', 'KS test']

df = pd.DataFrame(index=rows, columns=columns) 
for row in rows:
    for column in columns:
        if row in Dictionary_General and column in Dictionary_General[row]:
            dict_temp = Dictionary_General[row].get(column, [None])[nSamplesIdx] 
            df.loc[row, column] = dict_temp 
print('Sample size = ', OneSample, '| precision = ', precision, '| alpha = ', alpha)
print()
print(df)


file_path = Results_directory +'/'+ OneSample+'_'+precision+'_AllTests_table.txt'
print(file_path)
table = tabulate(df, headers='keys', tablefmt='pretty')

with open(file_path, 'w') as file:
    file.write(f"alpha = {alpha}\n")        
    file.write(f"{OneSample}, {precision} \n")        
    file.write(table)
    file.write('\n') 